In [2]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [13]:
import time
import numpy as np

In [5]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
x_train = x_train/255. 
x_test = x_test/255.

In [7]:
x_train = x_train.reshape(-1,28*28)
x_test = x_test.reshape(-1,28*28)

#Define non distributed model

In [8]:
model_normal = tf.keras.models.Sequential()
model_normal.add(tf.keras.layers.Dense(units=128,activation='relu',input_shape=(28*28,)))
model_normal.add(tf.keras.layers.Dropout(rate=0.2))
model_normal.add(tf.keras.layers.Dense(units=10,activation='softmax'))
model_normal.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])

#Distributed Strategy

In [9]:
distribute = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


# Distributed model defination

In [10]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128,activation='relu',input_shape=(28*28,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10,activation='softmax'))
  model_distributed.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])

#Comparison of both

In [11]:
start_time = time.time()
model_normal.fit(x_train,y_train,epochs=5,batch_size=32)
print("Normal model took {} seconds of time".format(time.time()-start_time))

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2942 - sparse_categorical_accuracy: 0.9158
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1465 - sparse_categorical_accuracy: 0.9567
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1070 - sparse_categorical_accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0880 - sparse_categorical_accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0763 - sparse_categorical_accuracy: 0.9758
Normal model took 20.169447660446167 seconds of time


In [12]:
start_time = time.time()
model_distributed.fit(x_train,y_train,epochs=5,batch_size=32)
print("Distributed model took {} seconds of time".format(time.time()-start_time))

Epoch 1/5
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tenso

Due to the combination of CPU and GPU we see the above results, as CPU worsen the performance of the GPU. But if you have two or more GPU's you can watch the distributed model works better